In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Загрузим таблицу с вопросами
df = pd.read_excel("questions.xlsx")  # должна содержать 'topic' и 'nli_label'

# Создаем комбинированную метку
df['strata'] = df['topic'].astype(str) + '|'  + '|' + df['correctness_response_nli'].astype(str)



In [11]:
strata_counts = df['strata'].value_counts()
print(strata_counts[strata_counts < 2])

strata
отчеты и выгрузки|згп|2                                 1
отчеты и выгрузки|инцидент|0                            1
работа продукта|кубики|2                                1
работа продукта|кубики|0                                1
отчеты и выгрузки|выгрузка|0                            1
перевод на специалиста|правило валидации|0              1
функционалы|PPD|1                                       1
отчеты и выгрузки|Отчеты|0                              1
функционалы|ВВП|0                                       1
функционалы|PPD|0                                       1
перевод в запрос|перевод в запрос|1                     1
инцидент- менеджмент|стоимость направления на ТП|0      1
перевод на специалиста|фикс продукты|1                  1
работа продукта|блокировки|0                            1
перевод в запрос|перевод  в запрос на тестирование|1    1
отчеты и выгрузки|работа продукта|1                     1
функционалы|источники обращения|0                       1
инциден

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Загрузите ваш файл с вопросами
df = pd.read_excel("questions.xlsx")   # Убедитесь, что есть колонки: 'topic' и 'nli_label'

# Создание страты из темы и метки
df['strata'] = df['topic'].astype(str) + '|' + df['correctness_response_nli'].astype(str)

# Подсчёт количества в каждой страте
strata_counts = df['strata'].value_counts()

# Выделение редких страт (меньше 2)
rare_strata = strata_counts[strata_counts < 2].index
rare_samples = df[df['strata'].isin(rare_strata)].copy()
rare_samples['is_rare'] = True

# Остальные вопросы (без редких)
df_remaining = df[~df.index.isin(rare_samples.index)].copy()
df_remaining['is_rare'] = False

# Сколько ещё нужно вопросов (чтобы в сумме было 50)
n_needed = 50 - len(rare_samples)

# Стратифицированная выборка из оставшихся
_, strat_sample = train_test_split(
    df_remaining,
    test_size=n_needed,
    stratify=df_remaining['strata'],
    random_state=42
)

# Объединение редких и стратифицированных
final_sample = pd.concat([rare_samples, strat_sample])
final_sample = final_sample.sample(frac=1, random_state=42).reset_index(drop=True)
final_sample=final_sample.merge(df[['initial_question','faithfulness_response_nli']], on='initial_question', how='left')

# Выгрузка в Excel
final_sample.to_excel("human_eval_sample_2004.xlsx", index=False)

print("Готово! Файл сохранён как human_eval_sample_2004.xlsx")

Готово! Файл сохранён как human_eval_sample_2004.xlsx


In [3]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score, matthews_corrcoef

# Предположим, данные уже загружены в DataFrame data
data = pd.read_excel('human_eval_sample.xlsx')

# Получаем список уникальных дополнительных экспертов
experts = data['expert_name'].unique()

results = []

for expert in experts:
    # выбираем данные для текущего эксперта
    df_expert = data[data['expert_name'] == expert]
    main_scores = df_expert['correctness_me']
    other_scores = df_expert['correctness_other']
    
    # считаем Cohen's Kappa
    kappa = cohen_kappa_score(main_scores, other_scores)
    
    # считаем MCC
    mcc = matthews_corrcoef(main_scores, other_scores)
    
    results.append({
        'pair': f'expert_main vs {expert}',
        'cohen_kappa': kappa,
        'mcc': mcc
    })

# создаем таблицу результатов
results_df = pd.DataFrame(results)

print(results_df)

# Средние значения
mean_kappa = results_df['cohen_kappa'].mean()
mean_mcc = results_df['mcc'].mean()

print(f'\nСредний Cohen\'s Kappa: {mean_kappa:.3f}')
print(f'Средний MCC: {mean_mcc:.3f}')

                     pair  cohen_kappa       mcc
0      expert_main vs Аня     1.000000  1.000000
1   expert_main vs Таня к     0.523810  0.523810
2  expert_main vs Ксюша С     0.615385  0.666667
3     expert_main vs Катя     0.444444  0.534522
4  expert_main vs Ксюша И     0.782609  0.801784

Средний Cohen's Kappa: 0.673
Средний MCC: 0.705


In [48]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score, matthews_corrcoef

# Предположим, данные уже загружены в DataFrame data
data = pd.read_excel('human_eval_sample.xlsx')

# Получаем список уникальных дополнительных экспертов
experts = data['expert_name'].unique()

results_correctness = []
results_hallucinations = []

for expert in experts:
    # выбираем данные для текущего эксперта
    df_expert = data[data['expert_name'] == expert]
    
    # Согласованность по корректности
    main_correctness_scores = df_expert['correctness_me']
    other_correctness_scores = df_expert['correctness_other']
    
    kappa_correctness = cohen_kappa_score(main_correctness_scores, other_correctness_scores)
    mcc_correctness = matthews_corrcoef(main_correctness_scores, other_correctness_scores)
    
    results_correctness.append({
        'pair': f'expert_main vs {expert}',
        'cohen_kappa': kappa_correctness,
        'mcc': mcc_correctness
    })
    
    # Согласованность по галлюцинациям
    main_hallucinations_scores = df_expert['halucinations_me']
    other_hallucinations_scores = df_expert['halucinations_other']
    
    kappa_hallucinations = cohen_kappa_score(main_hallucinations_scores, other_hallucinations_scores)
    mcc_hallucinations = matthews_corrcoef(main_hallucinations_scores, other_hallucinations_scores)
    
    results_hallucinations.append({
        'pair': f'expert_main vs {expert}',
        'cohen_kappa': kappa_hallucinations,
        'mcc': mcc_hallucinations
    })

# создаем таблицы результатов
results_df_correctness = pd.DataFrame(results_correctness)
results_df_hallucinations = pd.DataFrame(results_hallucinations)

print("Согласованность по корректности:")
print(results_df_correctness)

print("\nСогласованность по галлюцинациям:")
print(results_df_hallucinations)

# Средние значения для корректности
mean_kappa_correctness = results_df_correctness['cohen_kappa'].mean()
mean_mcc_correctness = results_df_correctness['mcc'].mean()

print(f'\nСредний Cohen\'s Kappa по корректности: {mean_kappa_correctness:.4f}')
print(f'Средний MCC по корректности: {mean_mcc_correctness:.4f}')

# Средние значения для галлюцинаций
mean_kappa_hallucinations = results_df_hallucinations['cohen_kappa'].mean()
mean_mcc_hallucinations = results_df_hallucinations['mcc'].mean()

print(f'\nСредний Cohen\'s Kappa по галлюцинациям: {mean_kappa_hallucinations:.4f}')
print(f'Средний MCC по галлюцинациям: {mean_mcc_hallucinations:.4f}')

Согласованность по корректности:
                     pair  cohen_kappa       mcc
0     expert_main vs Катя     0.444444  0.534522
1   expert_main vs Таня к     0.523810  0.523810
2      expert_main vs Аня     1.000000  1.000000
3  expert_main vs Ксюша И     0.782609  0.801784
4  expert_main vs Ксюша С     0.615385  0.666667

Согласованность по галлюцинациям:
                     pair  cohen_kappa       mcc
0     expert_main vs Катя     1.000000  1.000000
1   expert_main vs Таня к     1.000000  1.000000
2      expert_main vs Аня     1.000000  1.000000
3  expert_main vs Ксюша И     1.000000  1.000000
4  expert_main vs Ксюша С     0.782609  0.801784

Средний Cohen's Kappa по корректности: 0.6732
Средний MCC по корректности: 0.7054

Средний Cohen's Kappa по галлюцинациям: 0.9565
Средний MCC по галлюцинациям: 0.9604


In [50]:
data['correctness_nli'] = data['correctness_response_nli'].apply(lambda x: 1 if x == 0 else 0)

In [51]:
kappa = cohen_kappa_score(data['correctness_other'], data['correctness_nli'])
mcc = matthews_corrcoef(data['correctness_other'], data['correctness_nli'])

print(f"Cohen's Kappa: {kappa:.4f}")
print(f"MCC: {mcc:.4f}")

Cohen's Kappa: 0.5424
MCC: 0.5546


In [ ]:
kappa = cohen_kappa_score(data['correctness_me'], data['correctness_nli'])
mcc = matthews_corrcoef(data['correctness_me'], data['correctness_nli'])

print(f"Cohen's Kappa: {kappa:.4f}")
print(f"MCC: {mcc:.4f}")

Cohen's Kappa: 0.692
MCC: 0.693


In [35]:
import pandas as pd

# Загрузите основной DataFrame
data = pd.read_excel('human_eval_sample.xlsx')

# Загрузите дополнительный DataFrame из другого файла
additional_data = pd.read_excel('questions.xlsx')

# Объединяем два DataFrame на основе столбца 'initial_question'
# Используем how='inner' для исключения строк, где нет совпадений
merged_data = pd.merge(data, additional_data[['response', 'faithfulness_response_nli']], on='response', how='inner')

# Теперь merged_data содержит только строки с совпадениями из обоих DataFrame


In [36]:
merged_data['faithfulness_response_nli_0'] = merged_data['faithfulness_response_nli'].apply(lambda x: 1 if x == 2 else 0)

In [37]:
merged_data['faithfulness_response_nli_1'] = merged_data['faithfulness_response_nli'].apply(lambda x: 1 if x in [1, 2] else 0)

In [ ]:
#contradiction
kappa = cohen_kappa_score(merged_data['halucinations_me'], merged_data['faithfulness_response_nli_0'])
mcc = matthews_corrcoef(merged_data['halucinations_me'], merged_data['faithfulness_response_nli_0'])

print(f"Cohen's Kappa: {kappa:.4f}")
print(f"MCC: {mcc:.4f}")

Cohen's Kappa: -0.081
MCC: -0.082


In [ ]:
#contradiction
kappa = cohen_kappa_score(merged_data['halucinations_other'], merged_data['faithfulness_response_nli_0'])
mcc = matthews_corrcoef(merged_data['halucinations_other'], merged_data['faithfulness_response_nli_0'])

print(f"Cohen's Kappa: {kappa:.4f}")
print(f"MCC: {mcc:.4f}")

Cohen's Kappa: -0.098
MCC: -0.100


In [ ]:
#contradiction+neutral
kappa = cohen_kappa_score(merged_data['halucinations_me'], merged_data['faithfulness_response_nli_1'])
mcc = matthews_corrcoef(merged_data['halucinations_me'], merged_data['faithfulness_response_nli_1'])

print(f"Cohen's Kappa: {kappa:.4f}")
print(f"MCC: {mcc:.4f}")

Cohen's Kappa: 0.122
MCC: 0.161


In [ ]:
#contradiction+neutral
kappa = cohen_kappa_score(merged_data['halucinations_other'], merged_data['faithfulness_response_nli_1'])
mcc = matthews_corrcoef(merged_data['halucinations_other'], merged_data['faithfulness_response_nli_1'])

print(f"Cohen's Kappa: {kappa:.4}")
print(f"MCC: {mcc:.4f}")

Cohen's Kappa: 0.081
MCC: 0.103
